In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
import datetime as dt 
from datetime import datetime, timedelta, date

import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all

import xlsxwriter as xl

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [2]:
# Start included, end excluded.

#update 'start' date in this cell and also the file name in cell 70 & 71 (out of 73 cells)

last_month = dt.date.today().replace(day=1) - dt.timedelta(days=1)
start= dt.datetime.strptime("2017-01-01", "%Y-%m-%d")
end = start+timedelta(days=365)

query_vars = dict(
    snapshot = last_month.strftime("%Y-%m"),
    start= start.strftime('%Y-%m-%d'), 
    end = end.strftime('%Y-%m-%d'),
    pv_start = start.strftime("%Y%m"),
    pv_end = end.strftime("%Y%m"),
    
    ner_start = start.strftime("%Y-%m"),
    ner_end = end.strftime("%Y-%m"),
)

# List of wikis

In [3]:
# Gather all content wikis
wikis = wmf.hive.run("""
SELECT
  database_code,
  database_group AS project_code,
  language_code,
  CONCAT("https://", domain_name) AS domain_name,
  language_name,
  english_name as wiki_name
FROM canonical_data.wikis
WHERE
  database_group in (
    "commons", "incubator", "foundation", "mediawiki", "meta", "sources",
    "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
    "wikisource", "wikiversity", "wikivoyage", "wiktionary"
  ) AND
  status = "open" AND
  visibility = "public" AND
  editability = "public"
""")

In [4]:
wikis.sample(10)

,database_code,project_code,language_code,domain_name,language_name,wiki_name
647,tlwikibooks,wikibooks,tl,https://tl.wikibooks.org,Tagalog,Tagalog Wikibooks
54,bgwiktionary,wiktionary,bg,https://bg.wiktionary.org,Bulgarian,Bulgarian Wiktionary
619,szlwiki,wikipedia,szl,https://szl.wikipedia.org,Silesian,Silesian Wikipedia
258,hrwikibooks,wikibooks,hr,https://hr.wikibooks.org,Croatian,Croatian Wikibooks
516,ptwikivoyage,wikivoyage,pt,https://pt.wikivoyage.org,Portuguese,Portuguese Wikivoyage
171,euwiki,wikipedia,eu,https://eu.wikipedia.org,Basque,Basque Wikipedia
124,dewikisource,wikisource,de,https://de.wikisource.org,German,German Wikisource
226,gnwiki,wikipedia,gn,https://gn.wikipedia.org,Guarani,Guarani Wikipedia
597,srwikinews,wikinews,sr,https://sr.wikinews.org,Serbian,Serbian Wikinews
357,kuwiktionary,wiktionary,ku,https://ku.wiktionary.org,Kurdish,Kurdish Wiktionary


# Data

In [5]:
def merge_in(df, on="database_code"):
    global wikis
    wikis = pd.merge(wikis, df, how="left", on=on).fillna(0)
    
def top_10(df, col):
    return df.sort_values(col, ascending=False).head(10)
  
def rename_df(df):
    return df.rename({"wiki": "database_code", "domain": "domain_name"}, axis=1)

In [6]:
wikis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 6 columns):
database_code    732 non-null object
project_code     732 non-null object
language_code    732 non-null object
domain_name      732 non-null object
language_name    732 non-null object
wiki_name        732 non-null object
dtypes: object(6)
memory usage: 34.4+ KB


## Monthly active editors

In [7]:
mae = wmf.hive.run("""
SELECT
    wiki AS database_code,
    COUNT(*) / 12 AS monthly_active_editors,
    SUM(
        CAST(TRUNC(user_registration, 'MM') = TRUNC(month, 'MM') AS INT)
        )/ 12 AS monthly_new_active_editors
FROM cchen.editor_month
WHERE
    content_edits >= 5 
    AND month >= "{start}" 
    AND month < "{end}" 
    AND user_id != 0 
    --AND user_id IS NOT NULL
    AND bot_by_group = FALSE 
    AND (
        user_name not regexp "bot\\b" or
        user_name in ("Paucabot", "Niabot", "Marbot")    
    )    
GROUP BY wiki
""".format(**query_vars))

In [8]:
mae

,database_code,monthly_active_editors,monthly_new_active_editors
0,acewiki,2.916667,0.166667
1,adywiki,0.666667,0.083333
2,afwiki,34.333333,3.166667
3,afwiktionary,1.083333,0.250000
4,anwiktionary,0.083333,0.000000
...,...,...,...
672,zh_min_nanwiki,14.000000,0.416667
673,zh_yuewiki,96.583333,31.916667
674,zhwikiquote,8.083333,1.250000
675,zhwikisource,32.833333,2.333333


In [9]:
merge_in(mae)

## Monthly unique devices

In [10]:
mud = wmf.hive.run("""
SELECT
    regexp_replace(
        regexp_replace(
            regexp_replace(domain, "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.') AS domain_name,
  SUM(uniques_estimate) / 12 AS monthly_unique_devices
FROM wmf.unique_devices_per_domain_monthly
WHERE 
    CONCAT(year, month) >= "{pv_start}" and
    CONCAT(year, month) < "{pv_end}"
GROUP BY    
    regexp_replace(
        regexp_replace(
            regexp_replace(domain, "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.')
""".format(**query_vars))

mud["domain_name"] = "https://" + mud["domain_name"]

top_10(mud, "monthly_unique_devices")

,domain_name,monthly_unique_devices
194,https://en.wikipedia.org,6.964566e+08
208,https://es.wikipedia.org,1.398411e+08
164,https://de.wikipedia.org,9.296435e+07
366,https://ja.wikipedia.org,8.437400e+07
629,https://ru.wikipedia.org,8.284895e+07
249,https://fr.wikipedia.org,7.605284e+07
598,https://pt.wikipedia.org,4.991206e+07
356,https://it.wikipedia.org,4.746842e+07
853,https://zh.wikipedia.org,3.320957e+07
32,https://ar.wikipedia.org,2.859339e+07


In [11]:
merge_in(mud, on="domain_name")

## Overall SIZE rank

In [12]:
SIZE = np.sqrt(wikis["monthly_unique_devices"] * wikis["monthly_active_editors"])
rank = SIZE.rank(method="min", na_option="bottom", ascending=False)
wikis["overall_SIZE_rank"] = rank

## New editor retention

In [13]:
with open("queries/new_editor_retention.hql") as f:
    q = f.read()

ner = wmf.hive.run(
    q.format(start = "{ner_start}", end = "{ner_end}").format(**query_vars))

In [14]:
top_10(ner, "new_editor_retention")

,database_code,new_editor_retention
328,vowiki,1.0
17,bxrwiki,1.0
521,brwikisource,1.0
124,sgwiki,1.0
351,cdowiki,1.0
354,csbwiki,1.0
135,stqwiki,1.0
285,piwiki,1.0
280,nrmwiki,1.0
584,liwikiquote,1.0


In [15]:
merge_in(ner)

## Mobile editing proportion

In [16]:
mep = wmf.hive.run(
"""
SELECT 
    wiki AS database_code,
    SUM(mobile_web_edits + mobile_app_edits) / SUM(edits) AS mobile_editing_proportion
FROM cchen.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}"
    -- A user is a bot if they have a matching name or have the bot flag on *any* wiki
    -- See https://meta.wikimedia.org/wiki/Research:Active_editor and https://meta.wikimedia.org/wiki/Research:Bot_user
    AND user_id != 0
    AND NOT bot_by_group 
    AND (user_name not regexp "bot\\b" OR user_name in ("Paucabot", "Niabot", "Marbot"))
GROUP BY wiki
""".format(**query_vars))

In [17]:
top_10(mep, "mobile_editing_proportion")

,database_code,mobile_editing_proportion
176,rmywiki,0.708924
406,pswiki,0.506084
223,urwiktionary,0.361111
128,lmowiki,0.348367
122,lezwiki,0.337234
407,pswiktionary,0.318182
319,hawiktionary,0.268293
727,xmfwiki,0.250886
606,lowiki,0.230229
591,knwikisource,0.227607


In [18]:
merge_in(mep)

## Bot editing proportion

In [19]:
bep = wmf.hive.run("""
SELECT
   database_code,
   SUM(CASE WHEN user_is_bot THEN edit_count END)/ SUM(edit_count) AS bot_editing_proportion
FROM wmf.edit_hourly
INNER JOIN canonical_data.wikis ON CONCAT(project,".org") = domain_name
    AND database_group in 
    (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
WHERE
    ts  >= "{start}" 
   AND ts  < "{end}"
   AND snapshot = "{snapshot}"
GROUP BY database_code
""".format(**query_vars))

top_10(bep, "bot_editing_proportion")

,database_code,bot_editing_proportion
330,cebwiki,0.979345
352,rowiktionary,0.972799
257,cywiki,0.961482
52,iswiktionary,0.936898
22,nahwiktionary,0.918107
295,cawikinews,0.914482
373,eswiktionary,0.907897
2,bswikinews,0.891122
520,ruwiktionary,0.889169
721,zh_min_nanwiki,0.878936


In [20]:
merge_in(bep)

## Anonymous editing proportion

In [21]:
aep = wmf.hive.run("""
SELECT
   database_code,
   SUM(IF(user_is_anonymous, edit_count, 0)) / SUM(edit_count)  AS anonymous_editing_proportion
FROM wmf.edit_hourly
INNER JOIN canonical_data.wikis ON CONCAT(project,".org") = domain_name
    AND database_group in 
    (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
WHERE
    ts >= "{start}" 
   AND ts < "{end}"
   AND NOT user_is_bot
   AND snapshot = "{snapshot}"
GROUP BY database_code
""".format(**query_vars))

top_10(aep, "anonymous_editing_proportion")

,database_code,anonymous_editing_proportion
442,sqwikibooks,0.866883
284,tlwikibooks,0.839844
146,viwikibooks,0.759501
72,cowiki,0.694956
403,srwikiquote,0.662404
177,mgwiki,0.615800
531,zhwikiversity,0.609831
217,idwikiquote,0.585516
87,kowikibooks,0.584200
110,bgwikiquote,0.566733


In [22]:
merge_in(aep)

## Majority-mobile editors proportion

In [23]:
mmep = wmf.hive.run("""
SELECT 
   wiki AS database_code,
   SUM(CASE WHEN mobile_editing_proportion > 0.5 THEN 1 END) / COUNT(*) as majority_mobile_editors_proportion
FROM 
(
    SELECT 
        wiki,
        SUM(mobile_web_edits + mobile_app_edits) / SUM(edits) AS mobile_editing_proportion
    FROM cchen.editor_month
    WHERE
        month >= "{start}" 
        AND month < "{end}" 
        AND user_id != 0
        AND NOT bot_by_group 
        AND (user_name not regexp "bot\\b" OR user_name in ("Paucabot", "Niabot", "Marbot"))
    GROUP BY wiki, user_name
) user_edits
GROUP BY wiki
""".format(**query_vars))
top_10(mmep, "majority_mobile_editors_proportion")

,database_code,majority_mobile_editors_proportion
10,arwikisource,0.501085
62,fawiktionary,0.491443
8,arwikinews,0.411243
323,hiwiki,0.405701
22,bnwiki,0.399300
191,sowiki,0.395089
7,arwiki,0.391839
477,arwikibooks,0.383333
563,hiwikibooks,0.378788
9,arwikiquote,0.373288


In [24]:
merge_in(mmep)

## Revert rate

In [25]:
rr = wmf.hive.run("""
    SELECT
        wiki_db AS database_code,
        SUM(IF(revision_is_identity_reverted, 1, 0)) / COUNT(*) AS revert_rate
        --SUM(CAST(revision_is_identity_reverted AS INT)) / COUNT(*) AS revert_rate
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND event_timestamp >= "{start}" 
        AND event_timestamp < "{end}" 
        AND SIZE(event_user_is_bot_by_historical) = 0 
    GROUP BY wiki_db
""".format(**query_vars))

top_10(rr, "revert_rate")

,database_code,revert_rate
523,xalwiki,0.529227
456,mgwikibooks,0.375000
378,dzwiki,0.374280
579,tlwikibooks,0.362215
506,trwikinews,0.360515
31,nycwikimedia,0.360000
513,azbwiki,0.347966
391,ltwikisource,0.340426
508,etwikiquote,0.331797
779,viwikiquote,0.329538


In [26]:
merge_in(rr)

## Monthly pageviews

In [27]:
pageviews = hive.run("""
SELECT CONCAT("https://", project, ".org") AS domain_name, 
       SUM(view_count)/12 AS monthly_average_pageviews
FROM wmf.projectview_hourly
WHERE
    agent_type = "user" 
    AND CONCAT(year, month) >= "{pv_start}" 
    AND CONCAT(year, month) < "{pv_end}"
GROUP BY CONCAT("https://", project, ".org")
""".format(**query_vars))
top_10(pageviews, "monthly_average_pageviews")

,domain_name,monthly_average_pageviews
653,https://en.wikipedia.org,7.617446e+09
272,https://es.wikipedia.org,1.095616e+09
422,https://ja.wikipedia.org,1.055091e+09
20,https://de.wikipedia.org,9.752203e+08
99,https://ru.wikipedia.org,9.188346e+08
531,https://fr.wikipedia.org,6.865219e+08
50,https://it.wikipedia.org,5.132897e+08
497,https://zh.wikipedia.org,3.683159e+08
585,https://pt.wikipedia.org,3.379222e+08
206,https://pl.wikipedia.org,2.476481e+08


In [28]:
pageviews = pageviews.replace("https://wikidata.org", "https://www.wikidata.org")

In [29]:
merge_in(pageviews, on="domain_name")

## Mobile pageviews proportion

In [30]:
mpp = wmf.hive.run("""
SELECT 
    CONCAT("https://", project, ".org") AS domain_name,
    SUM(if(access_method = "mobile web", view_COUNT, 0)) / SUM(view_COUNT) AS mobile_web_pageviews_proportion,
    SUM(if(access_method = "mobile app", view_COUNT, 0)) / SUM(view_COUNT) AS mobile_app_pageviews_proportion
FROM wmf.projectview_hourly
WHERE
    agent_type = "user" 
    AND CONCAT(year, month) >= "{pv_start}" 
    AND CONCAT(year, month) < "{pv_end}" 
GROUP BY CONCAT("https://", project, ".org")
""".format(**query_vars))

In [31]:
mpp = mpp.replace("https://wikidata.org", "https://www.wikidata.org")

In [32]:
top_10(mpp, "mobile_web_pageviews_proportion")

,domain_name,mobile_web_pageviews_proportion,mobile_app_pageviews_proportion
620,https://wikipedia.org,1.000000,0.000000
668,https://hi.wikibooks.org,0.904597,0.000247
540,https://hi.wikipedia.org,0.864018,0.011292
550,https://jv.wiktionary.org,0.831347,0.000000
291,https://id.wikibooks.org,0.823915,0.000046
10,https://bn.wikipedia.org,0.813080,0.013498
44,https://hi.wikiquote.org,0.810344,0.000018
176,https://id.wiktionary.org,0.808276,0.000004
49,https://ig.wikipedia.org,0.763000,0.000078
545,https://id.wikiquote.org,0.751988,0.000398


In [33]:
merge_in(mpp, on="domain_name")

## Monthly active administrators

In [34]:
maa = hive.run("""
SELECT
    wiki as database_code,
    sum(monthly_active_administrators) / 12 as monthly_active_administrators
FROM (
    SELECT
        wiki_db as wiki,
        substr(log_timestamp, 1, 6) as month,
        count(distinct log_actor) as monthly_active_administrators
    from wmf_raw.mediawiki_logging
    WHERE
        log_type in ("block", "delete", "protect", "rights")
        -- Omit the "delete_redir", "move_prot", and "autopromote" actions, which can be done by regular users
        AND log_action not in ("autopromote", "delete_redir", "move_prot")
        AND log_timestamp >= "{start}" 
        AND log_timestamp < "{end}" 
        AND snapshot = "{snapshot}"
    GROUP BY wiki_db, substr(log_timestamp, 1, 6)
) mae
GROUP BY wiki
""".format(**query_vars))

top_10(maa, "monthly_active_administrators")

,database_code,monthly_active_administrators
618,enwiki,428.833333
307,commonswiki,166.833333
322,dewiki,137.750000
328,frwiki,106.083333
356,ruwiki,103.916667
97,itwiki,98.166667
353,ptwiki,85.166667
646,plwiki,74.166667
445,metawiki,55.750000
324,eswiki,52.833333


In [35]:
merge_in(maa)

## Monthly non-bot edits

In [36]:
mnbe = wmf.hive.run("""
SELECT
   database_code,
   SUM(edit_count) /12 AS monthly_nonbot_edits
FROM wmf.edit_hourly
INNER JOIN canonical_data.wikis ON CONCAT(project,".org") = domain_name
    AND database_group in 
    (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
WHERE
    ts >= "{start}" 
   AND ts < "{end}" 
   AND NOT user_is_bot
   AND snapshot = "{snapshot}" 
GROUP BY database_code
""".format(**query_vars))

top_10(mnbe, "monthly_nonbot_edits")

,database_code,monthly_nonbot_edits
601,wikidatawiki,6.553774e+06
117,enwiki,4.195626e+06
153,commonswiki,2.849098e+06
145,viwiki,8.574487e+05
206,dewiki,8.004705e+05
338,frwiki,7.170889e+05
574,eswiki,5.649752e+05
237,ruwiki,4.968741e+05
53,itwiki,4.370338e+05
545,jawiki,3.443171e+05


In [37]:
merge_in(mnbe)

## Edits Gini coefficient

In [38]:
user_edits = wmf.hive.run("""
    SELECT
        wiki_db AS wiki,
        COUNT(*) AS user_edits
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND event_timestamp >= "{start}" 
        AND event_timestamp < "{end}" 
        AND SIZE(event_user_is_bot_by_historical) = 0
        --event_user_is_bot_by_name = false 
        --array_contains(event_user_groups, "bot") = false
        --array_contains(event_user_is_bot_by, "NULL")= false
    GROUP BY event_user_id, wiki_db
""".format(**query_vars))

In [39]:
# FROM https://github.com/oliviaguest/gini
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # FROM:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array = array + 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))

In [40]:
egc = user_edits.groupby("wiki").apply(lambda g: gini(g["user_edits"].values)).reset_index()

In [41]:
egc.columns = ["database_code", "edits_Gini_coefficient"]

In [42]:
merge_in(egc)

## Monthly editors

In [43]:
me = wmf.hive.run("""
SELECT
    wiki AS database_code,
    COUNT(*) / 12 AS monthly_editors
FROM cchen.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}" 
    AND user_id != 0 
    AND bot_by_group = FALSE
    AND (user_name not regexp "bot\\b" or user_name in ("Paucabot", "Niabot", "Marbot"))
GROUP BY wiki
""".format(**query_vars))

top_10(me, "monthly_editors")

,database_code,monthly_editors
47,enwiki,135199.916667
274,commonswiki,35041.583333
516,dewiki,21083.333333
725,wikidatawiki,18176.166667
528,eswiki,17574.166667
546,frwiki,17181.500000
347,jawiki,13119.666667
657,ruwiki,11572.750000
344,itwiki,8711.083333
242,zhwiki,7623.166667


In [44]:
merge_in(me)

## Unique devices per editor

In [45]:
wikis["unique_devices_per_editor"] = wikis["monthly_unique_devices"] / wikis["monthly_editors"]

In [46]:
wikis = wikis.replace([np.inf], 0)

## Article COUNT

In [47]:
#wikis_list = wikis["wiki"].tolist()
wikis_list = wikis["database_code"].tolist()

In [48]:
#as of 09/19
wikis_list_not_working = ['alswiktionary', 'alswikibooks', 'alswikiquote', 'mowiki', 'mowiktionary']

In [49]:
wikis_list_clean = [x for x in wikis_list if x not in wikis_list_not_working]

In [50]:
ac = wmf.mariadb.run("""
SELECT
    database() AS database_code,
    ss_good_articles AS article_COUNT
FROM site_stats
""", wikis_list_clean)

In [51]:
top_10(ac, "article_COUNT")

,database_code,article_COUNT
703,wikidatawiki,69407132
93,commonswiki,56746040
151,enwiktionary,6168382
144,enwiki,5977770
405,mgwiktionary,5838983
86,cebwiki,5378778
609,svwiki,3744697
208,frwiktionary,3638569
120,dewiki,2371434
201,frwiki,2160811


In [52]:
merge_in(ac)

## Cumulative content edits

In [53]:
cce = wmf.hive.run("""
    SELECT
        wiki_db AS database_code,
        COUNT(*) AS cumulative_content_edits
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND page_namespace_is_content = true 
        AND SIZE(event_user_is_bot_by_historical) = 0 
        AND array_contains(event_user_groups, "bot") = false
    GROUP BY wiki_db
""".format(**query_vars))

In [54]:
top_10(cce, "cumulative_content_edits")

,database_code,cumulative_content_edits
658,enwiki,572410646
22,wikidatawiki,394816147
505,commonswiki,203251690
407,dewiki,117812613
586,frwiki,94080032
200,eswiki,80198999
50,ruwiki,62406706
71,jawiki,57450456
534,itwiki,56632560
517,zhwiki,32775489


In [55]:
merge_in(cce)

## Edits per content page

In [56]:
wikis["edits_per_content_page"] = wikis["cumulative_content_edits"] / wikis["article_COUNT"]

## Script direction

In [57]:
rtl_url = "https://noc.wikimedia.org/conf/dblists/rtl.dblist"
rtl_wikis = pd.Series(requests.get(rtl_url).text.split("\n"))
rtl = pd.DataFrame({"database_code": rtl_wikis, "script_direction": "right-to-left"})

merge_in(rtl)
wikis["script_direction"] = wikis["script_direction"].replace([0], "left-to-right")

## Monthly structured discussions messages

In [58]:
msdm = wmf.mariadb.run(
"""
SELECT
    rev_user_wiki AS database_code,
    COUNT(*) / 12 AS monthly_structured_discussions_messages
FROM flowdb.flow_revision
WHERE
    rev_change_type in ("new-post", "reply") 
    AND date_format(FROM_unixtime(
        (conv(substring(hex(rev_id), 1, 12), 16, 10) >> 2) / 1000),
        "%Y-%m-%d %H:%i:%S") >= "{start}" 
    AND date_format(FROM_unixtime(
        (conv(substring(hex(rev_id), 1, 12), 16, 10) >> 2) / 1000),
        "%Y-%m-%d %H:%i:%S") < "{end}"
GROUP BY rev_user_wiki
""".format(**query_vars), "wikishared")

top_10(msdm, "monthly_structured_discussions_messages")

,database_code,monthly_structured_discussions_messages
20,mediawikiwiki,3429.5000
9,frwiki,2814.5833
38,zhwiki,1582.8333
37,wikidatawiki,1121.5000
0,arwiki,886.7500
2,cawiki,736.1667
15,hewiki,411.1667
5,elwiki,238.8333
16,idwiki,205.1667
22,nowiki,177.4167


In [59]:
merge_in(msdm)

## Visual edits

In [60]:
ve = wmf.hive.run("""
SELECT 
    wiki AS database_code,
    SUM(visual_edits) / SUM(edits) AS visual_edits
FROM cchen.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}"
    AND user_id != 0
    AND NOT bot_by_group 
    AND (user_name not regexp "bot\\b" OR user_name in ("Paucabot", "Niabot", "Marbot"))
GROUP BY wiki
""".format(**query_vars))

top_10(ve, "visual_edits")

,database_code,visual_edits
219,ugwiki,0.636364
490,bgwikibooks,0.461187
173,ptwikiversity,0.369426
284,dinwiki,0.304718
594,kswiki,0.301829
356,kiwiki,0.256351
433,sqwikibooks,0.235294
102,kabwiki,0.224607
156,omwiki,0.214971
163,pihwiki,0.207700


In [61]:
merge_in(ve)

## Mobile unique devices

In [62]:
mob_ud = wmf.hive.run("""
SELECT
    regexp_replace(
        regexp_replace(
            regexp_replace(regexp_replace(domain, "www\\\\.", ""), "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.') AS domain_name,
    SUM(if((domain regexp '^m\\\\.' or  domain regexp '\\\\.m\\\\.'), uniques_estimate, 0)) AS mobile_COUNT,
    SUM(uniques_estimate) AS total_COUNT,
    SUM(
        IF((domain regexp '^m\\\\.' or  domain regexp '\\\\.m\\\\.'), uniques_estimate, 0)
    ) / SUM(uniques_estimate) AS mobile_unique_devices
FROM wmf.unique_devices_per_domain_monthly
WHERE 
    CONCAT(year, month) >= "{pv_start}" AND
    CONCAT(year, month) < "{pv_end}"
    
    
    
GROUP BY    
    regexp_replace(
        regexp_replace(
            regexp_replace(regexp_replace(domain, "www\\\\.", ""), "zero\\\\.", ""),
        '^m\\\\.', ''),
    '\\\\.m\\\\.', '.')
""".format(**query_vars))

mob_ud["domain_name"] = "https://" + mob_ud["domain_name"]

In [63]:
mob_ud = mob_ud.replace("https://wikidata.org", "https://www.wikidata.org")

In [64]:
top_10(mob_ud, "mobile_unique_devices")

,domain_name,mobile_count,total_count,mobile_unique_devices
340,https://ig.wikipedia.org,1576494,1659716,0.949858
299,https://hi.wikibooks.org,1385981,1495662,0.926667
304,https://hi.wiktionary.org,1184357,1317786,0.898748
300,https://hi.wikipedia.org,80130832,91233823,0.878302
375,https://jv.wiktionary.org,239642,274477,0.873086
642,https://sa.wiktionary.org,336046,385697,0.871269
301,https://hi.wikiquote.org,286321,330944,0.865164
91,https://bn.wikisource.org,527266,613176,0.859893
334,https://id.wikiquote.org,485423,575474,0.843519
332,https://id.wikibooks.org,5630417,6683891,0.842386


In [65]:
merge_in(mob_ud, on="domain_name")

# Readying for spreadsheet

In [66]:
wikis.columns.tolist()

['database_code',
 'project_code',
 'language_code',
 'domain_name',
 'language_name',
 'wiki_name',
 'monthly_active_editors',
 'monthly_new_active_editors',
 'monthly_unique_devices',
 'overall_SIZE_rank',
 'new_editor_retention',
 'mobile_editing_proportion',
 'bot_editing_proportion',
 'anonymous_editing_proportion',
 'majority_mobile_editors_proportion',
 'revert_rate',
 'monthly_average_pageviews',
 'mobile_web_pageviews_proportion',
 'mobile_app_pageviews_proportion',
 'monthly_active_administrators',
 'monthly_nonbot_edits',
 'edits_Gini_coefficient',
 'monthly_editors',
 'unique_devices_per_editor',
 'article_COUNT',
 'cumulative_content_edits',
 'edits_per_content_page',
 'script_direction',
 'monthly_structured_discussions_messages',
 'visual_edits',
 'mobile_count',
 'total_count',
 'mobile_unique_devices']

In [67]:
wikis.head()

,database_code,project_code,language_code,domain_name,language_name,wiki_name,monthly_active_editors,monthly_new_active_editors,monthly_unique_devices,overall_SIZE_rank,...,unique_devices_per_editor,article_COUNT,cumulative_content_edits,edits_per_content_page,script_direction,monthly_structured_discussions_messages,visual_edits,mobile_count,total_count,mobile_unique_devices
0,abwiki,wikipedia,ab,https://ab.wikipedia.org,Abkhazian,Abkhazian Wikipedia,2.000000,0.083333,9795.166667,406.0,...,721.116564,5976,26656.0,4.460509,left-to-right,0.0,0.001132,17675.0,117542.0,0.150372
1,acewiki,wikipedia,ace,https://ace.wikipedia.org,Achinese,Achinese Wikipedia,2.916667,0.166667,19162.166667,320.0,...,1116.242718,10298,40842.0,3.966013,left-to-right,0.0,0.004627,112454.0,229946.0,0.489045
2,adywiki,wikipedia,ady,https://ady.wikipedia.org,Adyghe,Adyghe Wikipedia,0.666667,0.083333,5189.583333,522.0,...,457.904412,415,5228.0,12.597590,left-to-right,0.0,0.007426,6673.0,62275.0,0.107154
3,afwiki,wikipedia,af,https://af.wikipedia.org,Afrikaans,Afrikaans Wikipedia,34.333333,3.166667,416171.916667,88.0,...,3061.963826,86840,862526.0,9.932358,left-to-right,0.0,0.053471,2898540.0,4994063.0,0.580397
4,afwikibooks,wikibooks,af,https://af.wikibooks.org,Afrikaans,Afrikaans Wikibooks,0.000000,0.000000,1103.666667,665.0,...,827.750000,23,680.0,29.565217,left-to-right,0.0,0.000000,3567.0,13244.0,0.269330


In [68]:
wikis = wikis[[
    'overall_SIZE_rank',
    'monthly_unique_devices',
    'mobile_unique_devices',
    'mobile_web_pageviews_proportion',
    'mobile_app_pageviews_proportion',
    'unique_devices_per_editor',
    'monthly_editors',
    'monthly_active_editors',
    'monthly_active_administrators',
    'majority_mobile_editors_proportion',
    'monthly_new_active_editors',
    'new_editor_retention',
    'monthly_nonbot_edits',
    'bot_editing_proportion',
    'mobile_editing_proportion',
    'visual_edits',
    'anonymous_editing_proportion',
    'revert_rate',
    'edits_Gini_coefficient',
    'monthly_structured_discussions_messages',
    'article_COUNT',
    'cumulative_content_edits',
    'edits_per_content_page',
    'script_direction',
    'database_code',
    'project_code',
    'language_code',
    'domain_name',
    'language_name',
    'project_code',
    'wiki_name',
]]

In [69]:
wikis.rename(columns={
    'article_COUNT':'content_pages',
    'anonymous_editing_proportion': 'anonymous_edits',
    'mobile_editing_proportion': 'mobile_edits',
    'bot_editing_proportion':'bot_edits',
    'new_editor_retention':'second_month_editor_retention',
    'majority_mobile_editors_proportion':'majority_mobile_editors',
    'mobile_app_pageviews_proportion':'mobile_app_pageviews',
    'mobile_web_pageviews_proportion':'mobile_web_pageviews',
    'domain_name':'domain',
    'wiki_name': 'wiki'
}, inplace=True)

#drop the underscore in headers
wikis.columns = wikis.columns.str.replace('_', ' ')

#add url column and put at the front
wikis['url'] = wikis['wiki'] + '#' + wikis['domain']
cols = wikis.columns.tolist()
cols = cols[-1:] + cols[:-1]
wikis = wikis[cols]

#sort by size
wikis = wikis.sort_values("overall SIZE rank").fillna(0).reset_index(drop=True)

#format to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format
#format floats to use comma separators https://pandas.pydata.org/pandas-docs/version/0.23.4/options.html
pd.options.display.float_format = '{:,}'.format

In [70]:
def make_hyperlink(val):
    wiki, domain = val.split('#')
    #url = "https://custom.url/{}"
    return '=HYPERLINK("{domain}", "{wiki}")'.format(domain=domain, wiki=wiki)

wikis['url'] = wikis['url'].apply(lambda x: make_hyperlink(x))

In [71]:
#write to xls
#https://xlsxwriter.readthedocs.io/working_with_pandas.html
#https://xlsxwriter.readthedocs.io/example_pandas_column_formats.html
    
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('wikis17.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
wikis.to_excel(writer, sheet_name='2017', float_format = "%0.2f", index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['2017']

# create desired xlsxwriter formats
headers = workbook.add_format({'bold': True})
alignment = workbook.add_format({'align': 'left'})

# apply formats to header and index
worksheet.set_row(0, None, headers)
worksheet.set_column(0,0, 18, alignment)
#set_column(first_col, last_col, width, cell_format, options)

# Add some cell formats
percent = workbook.add_format({'num_format': '0%'})
comma = workbook.add_format({'num_format': '#,##0.00'})


# Set the format on the percent columns.
# Set the format but not the column width.
worksheet.set_column('D:F', 18, percent)
worksheet.set_column('K:K', 18, percent)
worksheet.set_column('M:M', 18, percent)
worksheet.set_column('O:S', 18, percent)

worksheet.set_column('C:C', 18, comma)
worksheet.set_column('G:I', 18, comma)
worksheet.set_column('L:L', 18, comma)
worksheet.set_column('N:N', 18, comma)
worksheet.set_column('U:W', 18, comma)


# Add a sample alternative link format.
blue_format = workbook.add_format({
    'font_color': 'blue',
    'bold':       1,
    'underline':  1,
    'font_size':  12,
})

#Set the format and the width
worksheet.set_column('A:A', 50, blue_format)

worksheet.set_column('B:AF', 18)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [72]:
wikis.to_csv("wikis17.csv", sep=',', encoding = 'utf-8', index=False)

In [74]:
wikis.head()

,url,overall SIZE rank,monthly unique devices,mobile unique devices,mobile web pageviews,mobile app pageviews,unique devices per editor,monthly editors,monthly active editors,monthly active administrators,...,script direction,database code,project code,project code,language code,domain,language name,project code,project code,wiki
0,"=HYPERLINK(""https://en.wikipedia.org"", ""Englis...",1.0,"696,456,562.25",0.6407751718091207,0.47153478739910826,0.017958346404315047,"5,151.309108918337","135,199.91666666666","31,967.166666666668",428.8333333333333,...,left-to-right,enwiki,wikipedia,wikipedia,en,https://en.wikipedia.org,English,wikipedia,wikipedia,English Wikipedia
1,"=HYPERLINK(""https://es.wikipedia.org"", ""Spanis...",2.0,"139,841,119.75",0.6385768869436321,0.5669121644862457,0.008122724472246552,"7,957.197766608184","17,574.166666666668","4,279.0",52.833333333333336,...,left-to-right,eswiki,wikipedia,wikipedia,es,https://es.wikipedia.org,Spanish,wikipedia,wikipedia,Spanish Wikipedia
2,"=HYPERLINK(""https://de.wikipedia.org"", ""German...",3.0,"92,964,352.41666667",0.543703202027271,0.4191160589240039,0.05028997215577481,"4,409.376399209486","21,083.333333333332","5,648.0",137.75,...,left-to-right,dewiki,wikipedia,wikipedia,de,https://de.wikipedia.org,German,wikipedia,wikipedia,German Wikipedia
3,"=HYPERLINK(""https://ja.wikipedia.org"", ""Japane...",4.0,"84,374,000.08333333",0.6858784018320431,0.5739977340869467,0.009355990913711514,"6,431.108520287609","13,119.666666666666","4,507.583333333333",37.083333333333336,...,left-to-right,jawiki,wikipedia,wikipedia,ja,https://ja.wikipedia.org,Japanese,wikipedia,wikipedia,Japanese Wikipedia
4,"=HYPERLINK(""https://fr.wikipedia.org"", ""French...",5.0,"76,052,837.0",0.5717709408613734,0.4724599775157904,0.018289548115361206,"4,426.437563658586","17,181.5","4,748.916666666667",106.08333333333333,...,left-to-right,frwiki,wikipedia,wikipedia,fr,https://fr.wikipedia.org,French,wikipedia,wikipedia,French Wikipedia
